In [9]:
import tensorflow as tf
import numpy as np
import pandas as pd

ANN Architecture Summary

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

model = Sequential([
    layers.Input(shape = (128,)),
    layers.Normalization(mean = 127, variance = 77.4),
    layers.Dense(200, activation='tanh'), #86
    layers.Dense(160, activation='tanh'), #64
    layers.Dense(140, activation='tanh'), #42
    layers.Dense(100, activation='tanh'), #21
    layers.Dense(60, activation='tanh'),
    layers.Dense(1, activation='relu')
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization_2 (Normalizati (None, 128)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 200)               25800     
_________________________________________________________________
dense_13 (Dense)             (None, 160)               32160     
_________________________________________________________________
dense_14 (Dense)             (None, 140)               22540     
_________________________________________________________________
dense_15 (Dense)             (None, 100)               14100     
_________________________________________________________________
dense_16 (Dense)             (None, 60)                6060      
_________________________________________________________________
dense_17 (Dense)             (None, 1)                

Train and Save to Disk

In [18]:
save_to_disk = True # Set to FALSE if we don't want this to overwrite the current model.

TrainFeatures = pd.read_csv('../data/trainFeaturesNewSplit.csv', header = 0, index_col = 0).drop(['video_id','time_stamp'], axis = 'columns')
TrainTargets = pd.read_csv('../data/trainTargetsNewSplit.csv', header = 0, index_col = 0).drop(['video_id','time_stamp'], axis = 'columns')
TrainTargets.iloc[:,0] = TrainTargets.iloc[:,0].astype(float)

EvalFeatures = pd.read_csv('../data/evalFeaturesNewSplit.csv', header = 0, index_col = 0).drop(['video_id','time_stamp'], axis = 'columns')
EvalTargets = pd.read_csv('../data/evalTargetsNewSplit.csv', header = 0, index_col = 0).drop(['video_id','time_stamp'], axis = 'columns')
EvalTargets.iloc[:,0] = EvalTargets.iloc[:,0].astype(float)

# Since our dataset is imbalanced, we need a weighted binary cross entropy loss function.
def weighted_bce(y_true, y_pred):
    weights = (y_true * 8) + 1.
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    weighted_bce = tf.math.reduce_mean(bce * weights)
    return weighted_bce

# Here, we choose the loss function to minimize, the optimizer algorithm, and additional loss metrics to display on screen.
#model.compile(loss = weighted_bce, optimizer = 'adam', metrics = ['accuracy', tf.keras.metrics.AUC(name='auc')])
model.compile(loss = weighted_bce, optimizer = 'adam', metrics = ['accuracy'])

# Time to train! Epochs = # times the training algorithm goes through the whole set.
# Batch size = # elements the algorithm "sees" at a time between adjusting model parameters.
model.fit(TrainFeatures,TrainTargets, epochs = 10, batch_size = 8)

if save_to_disk:
    # Saving model to JSON and weights to H5.
    model_json = model.to_json()
    with open("../models/model.json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights("../models/model.h5")
    print("Saved model to disk")

# Measure accuracy on the evaluation set.
loss  = model.evaluate(EvalFeatures, EvalTargets)
print('Loss on Validation Set: ', loss[0])
print('Accuracy on Validation Set: ', loss[1])
predictions = np.round(model.predict(EvalFeatures))
predictions = np.minimum(predictions, np.ones((len(predictions),1))) # Sometimes the result can get rounded to 2.
true_labels = EvalTargets.iloc[:,0]

# Additional measurements that account for the set being unbalanced.
r = tf.keras.metrics.Recall()
r.update_state(true_labels, predictions)
print('Recall: ' + str(r.result().numpy()))

p = tf.keras.metrics.Precision()
p.update_state(true_labels, predictions)
print('Precision: ' + str(p.result().numpy()))

f1 = 2* r.result().numpy() * p.result().numpy() / (r.result().numpy() + p.result().numpy())
print('F1 Score: ' + str(f1))

print('Confusion matrix:')
conf_matrix = tf.math.confusion_matrix(true_labels, predictions)
conf_matrix

Epoch 1/10
7993/7993 [==============================] - 9s 1ms/step - loss: 1.4207 - accuracy: 0.8166
Epoch 2/10
7993/7993 [==============================] - 9s 1ms/step - loss: 1.1086 - accuracy: 0.8306
Epoch 3/10
7993/7993 [==============================] - 8s 1ms/step - loss: 2.4869 - accuracy: 0.8064
Epoch 4/10
7993/7993 [==============================] - 8s 1ms/step - loss: 1.3078 - accuracy: 0.8432
Epoch 5/10
7993/7993 [==============================] - 9s 1ms/step - loss: 1.6235 - accuracy: 0.8293
Epoch 6/10
7993/7993 [==============================] - 8s 1ms/step - loss: 1.5986 - accuracy: 0.7921
Epoch 7/10
7993/7993 [==============================] - 8s 1ms/step - loss: 1.5382 - accuracy: 0.8026
Epoch 8/10
7993/7993 [==============================] - 9s 1ms/step - loss: 1.0193 - accuracy: 0.8438
Epoch 9/10
7993/7993 [==============================] - 9s 1ms/step - loss: 2.6136 - accuracy: 0.8335
Epoch 10/10
7993/7993 [==============================] - 10s 1ms/step - loss: 2.45

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[9361,  864],
       [ 746, 1818]])>

Train Repeatedly to Generate Performance Statistics

In [19]:
number_of_trials = 30
accuracy = np.zeros(number_of_trials)
recall = np.zeros(number_of_trials)
precision = np.zeros(number_of_trials)
f1score = np.zeros(number_of_trials)

for i in range(0,number_of_trials):
    model.fit(TrainFeatures,TrainTargets, epochs = 10, batch_size = 8)

    # Measure accuracy on the evaluation set.
    loss  = model.evaluate(EvalFeatures, EvalTargets)
    accuracy[i] = loss[1]
    predictions = np.round(model.predict(EvalFeatures))
    predictions = np.minimum(predictions, np.ones((len(predictions),1))) # Sometimes the result can get rounded to 2.
    true_labels = EvalTargets.iloc[:,0]

    r = tf.keras.metrics.Recall()
    r.update_state(true_labels, predictions)
    recall[i] = r.result().numpy()
    
    p = tf.keras.metrics.Precision()
    p.update_state(true_labels, predictions)
    precision[i] = p.result().numpy()
    
    f1score[i] = 2* r.result().numpy() * p.result().numpy() / (r.result().numpy() + p.result().numpy())

print('Accuracy:')
print('Mean: ' + str(np.mean(accuracy)))
print('Std Dev: ' + str(np.std(accuracy)))
print(accuracy)
print('Recall:')
print('Mean: ' + str(np.mean(recall)))
print('Std Dev: ' + str(np.std(recall)))
print(recall)
print('Precision:')
print('Mean: ' + str(np.mean(precision)))
print('Std Dev: ' + str(np.std(precision)))
print(precision)
print('F1 Score:')
print('Mean: ' + str(np.mean(f1score)))
print('Std Dev: ' + str(np.std(f1score)))
print(f1score)

Epoch 1/10
7993/7993 [==============================] - 9s 1ms/step - loss: 3.5907 - accuracy: 0.7981
Epoch 2/10
7993/7993 [==============================] - 9s 1ms/step - loss: 1.4996 - accuracy: 0.8296
Epoch 3/10
7993/7993 [==============================] - 9s 1ms/step - loss: 1.3157 - accuracy: 0.8462
Epoch 4/10
7993/7993 [==============================] - 9s 1ms/step - loss: 1.2041 - accuracy: 0.8470
Epoch 5/10
7993/7993 [==============================] - 8s 1ms/step - loss: 1.1539 - accuracy: 0.8472
Epoch 6/10
7993/7993 [==============================] - 8s 1ms/step - loss: 1.2136 - accuracy: 0.8439
Epoch 7/10
7993/7993 [==============================] - 9s 1ms/step - loss: 2.7757 - accuracy: 0.8095
Epoch 8/10
7993/7993 [==============================] - 9s 1ms/step - loss: 1.5067 - accuracy: 0.8322
Epoch 9/10
7993/7993 [==============================] - 9s 1ms/step - loss: 2.8186 - accuracy: 0.7933
Epoch 10/10
400/400 [==============================] - 0s 778us/step - loss: 2.791

7993/7993 [==============================] - 8s 1ms/step - loss: 0.9382 - accuracy: 0.8562
Epoch 6/10
7993/7993 [==============================] - 8s 1ms/step - loss: 2.5102 - accuracy: 0.8053
Epoch 7/10
7993/7993 [==============================] - 8s 1ms/step - loss: 2.1312 - accuracy: 0.8117
Epoch 8/10
7993/7993 [==============================] - 8s 1ms/step - loss: 1.9887 - accuracy: 0.8197
Epoch 9/10
7993/7993 [==============================] - 8s 1ms/step - loss: 2.1438 - accuracy: 0.8066
Epoch 10/10
400/400 [==============================] - 0s 752us/step - loss: 2.4065 - accuracy: 0.2071
Epoch 1/10
7993/7993 [==============================] - 8s 1ms/step - loss: 1.2096 - accuracy: 0.8363
Epoch 2/10
7993/7993 [==============================] - 8s 1ms/step - loss: 1.2937 - accuracy: 0.8454
Epoch 3/10
7993/7993 [==============================] - 8s 1ms/step - loss: 2.4003 - accuracy: 0.8002
Epoch 4/10
7993/7993 [==============================] - 8s 1ms/step - loss: 1.8309 - accurac

7993/7993 [==============================] - 9s 1ms/step - loss: 5.3981 - accuracy: 0.6321
Epoch 10/10
400/400 [==============================] - 0s 727us/step - loss: 4.0839 - accuracy: 0.7442
Epoch 1/10
7993/7993 [==============================] - 9s 1ms/step - loss: 1.7654 - accuracy: 0.8389
Epoch 2/10
7993/7993 [==============================] - 9s 1ms/step - loss: 5.6890 - accuracy: 0.7275
Epoch 3/10
7993/7993 [==============================] - 9s 1ms/step - loss: 3.8169 - accuracy: 0.7489
Epoch 4/10
7993/7993 [==============================] - 9s 1ms/step - loss: 3.0062 - accuracy: 0.7884
Epoch 5/10
7993/7993 [==============================] - 9s 1ms/step - loss: 3.4522 - accuracy: 0.7650
Epoch 6/10
7993/7993 [==============================] - 9s 1ms/step - loss: 1.2819 - accuracy: 0.8490
Epoch 7/10
7993/7993 [==============================] - 9s 1ms/step - loss: 1.5352 - accuracy: 0.8398
Epoch 8/10
7993/7993 [==============================] - 9s 1ms/step - loss: 1.3703 - accurac

7993/7993 [==============================] - 9s 1ms/step - loss: 1.2028 - accuracy: 0.8539
Epoch 3/10
7993/7993 [==============================] - 9s 1ms/step - loss: 1.1140 - accuracy: 0.8502
Epoch 4/10
7993/7993 [==============================] - 9s 1ms/step - loss: 1.8723 - accuracy: 0.8249
Epoch 5/10
7993/7993 [==============================] - 9s 1ms/step - loss: 1.1868 - accuracy: 0.8511
Epoch 6/10
7993/7993 [==============================] - 9s 1ms/step - loss: 1.7974 - accuracy: 0.8404
Epoch 7/10
7993/7993 [==============================] - 8s 1ms/step - loss: 2.0486 - accuracy: 0.8372
Epoch 8/10
7993/7993 [==============================] - 8s 1ms/step - loss: 1.5417 - accuracy: 0.8383
Epoch 9/10
7993/7993 [==============================] - 9s 1ms/step - loss: 1.2434 - accuracy: 0.8538
Epoch 10/10
400/400 [==============================] - 0s 695us/step - loss: 1.3760 - accuracy: 0.8702
Epoch 1/10
7993/7993 [==============================] - 9s 1ms/step - loss: 1.8681 - accurac

7993/7993 [==============================] - 9s 1ms/step - loss: 2.1390 - accuracy: 0.8214
Epoch 7/10
7993/7993 [==============================] - 9s 1ms/step - loss: 1.1597 - accuracy: 0.8447
Epoch 8/10
7993/7993 [==============================] - 9s 1ms/step - loss: 1.3547 - accuracy: 0.8550
Epoch 9/10
7993/7993 [==============================] - 9s 1ms/step - loss: 1.3235 - accuracy: 0.8559
Epoch 10/10
400/400 [==============================] - 0s 752us/step - loss: 1.0041 - accuracy: 0.8769
Accuracy:
Mean: 0.8217687005798022
Std Dev: 0.16868136125602262
[0.84212995 0.86785519 0.87997496 0.88036597 0.87543982 0.8394714
 0.87692547 0.20713113 0.87317228 0.87942761 0.18797404 0.87598717
 0.87434512 0.87692547 0.74423331 0.86535305 0.87129563 0.88028777
 0.86918443 0.87637812 0.87606537 0.85565722 0.87020093 0.87731642
 0.86707324 0.87716007 0.87786376 0.87387598 0.85706466 0.87692547]
Recall:
Mean: 0.6599583978454272
Std Dev: 0.1200151271993405
[0.5370515  0.68174726 0.69890797 0.6712